# Create Prompts

In [9]:
import json
import pandas as pd
from datetime import datetime, timedelta
import os

# Constants
tickers = ['AMZN', 'JNJ', 'MSFT']
base_path = './Weekly-Closing-Prices/{}_Pricing/{}_weekly_close.json'
scenarios = {
    "Covid": "2020-03-13",
    "April Stable": "2019-07-12",
    "Drop": "2022-04-08"
}

# Function to load and filter the data
def load_and_filter_data(ticker, end_date):
    file_path = base_path.format(ticker, ticker)
    with open(file_path, 'r') as file:
        data = json.load(file)
    
    # Convert to DataFrame
    df = pd.DataFrame(data.items(), columns=['Date', 'Price'])
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Filter data up to the specified end date
    filtered_df = df[df['Date'] <= pd.to_datetime(end_date)].sort_values(by='Date')
    
    # Get the last 3 months of data
    start_date = pd.to_datetime(end_date) - timedelta(days=90)
    last_3_months_df = filtered_df[filtered_df['Date'] > start_date]
    
    return last_3_months_df

# Function to generate the prediction prompt
def generate_prompt(ticker, df):
    prompt = (f"Given the following information, stock closing prices for the past 3 months, "
              f"predict the next 12 weeks of closing prices for {ticker}. The predictions should be returned "
              f"in a date:price format, similar to the historical closing prices provided. "
              f"Ensure each prediction is on a new line and follows the exact format as shown in the examples (YYYY-MM-DD: $price).\n\n"
              f"- Stock Closing Prices for Past 3 Months (Fridays):")
    
    for date, price in zip(df['Date'].dt.strftime('%Y-%m-%d'), df['Price']):
        price_float = float(price)  # Convert price to float
        prompt += f"\n  - {date}: ${price_float:.4f}"
    
    prompt += ("\n\nPredict the next 12 weeks of closing prices for {ticker}, starting from the week following the last provided date. "
               "Use your understanding of market trends, sentiment analysis, and the provided data to inform your predictions.")
    
    return prompt

# Excel workbook setup
output_dir = './LLM-Prompt-Dataset'
os.makedirs(output_dir, exist_ok=True)
writer = pd.ExcelWriter(f'{output_dir}/stock_predictions.xlsx', engine='xlsxwriter')

# Generate data and write to Excel
for ticker in tickers:
    all_prompts = []
    for scenario_name, end_date in scenarios.items():
        df = load_and_filter_data(ticker, end_date)
        prompt = generate_prompt(ticker, df)
        all_prompts.append({'Scenario': f'{ticker} {scenario_name}', 'Prompt': prompt})
    
    # Convert to DataFrame and write to Excel
    scenario_df = pd.DataFrame(all_prompts)
    scenario_df.to_excel(writer, sheet_name=ticker, index=False)

# Close the Excel file
writer.close()


# Fetch Results from prompts

In [12]:
import json
import pandas as pd
import re
import os

# Constants
excel_path = './LLM-Prompt-Dataset/stock_predictions.xlsx'
base_output_path = '../outputs/{}/{}'

# Function to process the output text and extract predictions
def extract_predictions(output_text):
    pattern = re.compile(r'(\d{4}-\d{2}-\d{2}): \$([0-9]+\.?[0-9]*)')
    predictions = pattern.findall(output_text)
    return {date: price for date, price in predictions}

# Function to save predictions as a JSON file
def save_predictions_as_json(predictions, ticker, scenario):
    # Create the directory if it doesn't exist
    output_path = base_output_path.format(ticker, scenario)
    os.makedirs(output_path, exist_ok=True)

    # Save to JSON file
    json_file_path = os.path.join(output_path, f"LLM_prediction.json")
    with open(json_file_path, 'w') as json_file:
        json.dump(predictions, json_file, indent=4)

# Read the Excel file
excel_data = pd.ExcelFile(excel_path)

# Process each sheet in the Excel file
for sheet_name in excel_data.sheet_names:
    df = pd.read_excel(excel_data, sheet_name=sheet_name)
    for index, row in df.iterrows():
        if pd.notna(row['output']):  # Check if there is an output to process
            predictions = extract_predictions(row['output'])
            scenario = row['Scenario'].split()[-1]  # Assumes scenario is the last word in the Scenario column
            save_predictions_as_json(predictions, sheet_name, scenario)
            print(f"Saved predictions for {sheet_name} - {scenario}")

print("All predictions processed and saved.")


Saved predictions for AMZN - Covid
Saved predictions for AMZN - Stable
Saved predictions for AMZN - Drop
Saved predictions for JNJ - Covid
Saved predictions for JNJ - Stable
Saved predictions for JNJ - Drop
Saved predictions for MSFT - Covid
Saved predictions for MSFT - Stable
Saved predictions for MSFT - Drop
All predictions processed and saved.
